<a href="https://colab.research.google.com/github/DerekSHAOZH/FUB-22Win-Phylogeny-Inference-and-Application/blob/main/Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Problem 1**: 50 Points

#### A) (5 P) Write two classes: class Vertex and class Tree, that can be used as data structures to represent trees.

Add appropriate attributes to each class. For example, "name", "degree", "parent", "children", ... for the vertex class and "nodes", "root", ... for the tree class.

These classes will be the basis of the current and future exercises. In general, you have the freedom to implement the functions the way you see fit. Just keep in mind that you will use this class in each week and will have to continuously add functions (and also attributes) for the exercises.

#### B) (15 P) Add following functions to your class "Tree":

* AddVertex(self, vertex): Adds a vertex to your tree. It's up to you whether you want to use strings that contain the vertex name(s) or an object of the class vertex as input for the functions
* AddEdge(self, parent_vertex, child_vertex, distance): Which adds a directed, weighted edge from the parent to the child. It's up to you whether you want to use strings that contain the vertex name(s) or an object of the class vertex as input for the functions.
* ReadNewickFile(self,file_name): Read in a file containing a newick formatted string 


#### C) (10 P) Build following tree using your data structure: Directed edges = $\{(h_1, l_1), (h_1, l_2), (h_3, h_1), (h_3, l_3), (h_2, l_4), (h_2, l_5), (h_4, h_3), (h_4, h_2)\}$ and branch lengths = $(0.1, 0.3, 0.1, 0.5, 0.2, 0.2, 0.6, 0.3)$. Print the name of each vertex and its respective degree, in-degree and out-degree. Which vertex is the root?

#### D) (10 P) Add the function to your tree class ComputeNewickFormat(self) to generate a string that represents the current tree structure in the newick format. Use your function to generate the newick presentation for the tree build using the directed edges in C and print the generated string.

####E) (10 P) Add a function to your tree class ReadNewickFile(self, newickFile) that builds the tree given a newick file. Use this function to read the file named **tree.nwk**. Print the in-degree and out-degree of each vertex.

_Hint: You may want to use regular expressions to identify siblings, and replace siblings with a hidden vertex._

**Don't forget to comment your code!**



In [ ]:
import re
# 1 A and B class definitions and functions
class Vertex:
  def __init__(self,name): 
    self.name = name
    self.in_degree = 0
    self.out_degree = 0
    self.parent = self
    self.children = []
    self.neighbors = []
    self.newick_label = ""

class Tree:
  def __init__(self,name):
    self.name = name
    self.vertex_map = {}
    self.leaf_list = []
    self.pre_order_list = []
    self.post_order_list = []
    self.edge_list_map = {}
    self.root = -1
  def Add_vertex(self,name):    
    v = Vertex(name)
    self.vertex_map[name] = v  
  def Contains_vertex(self,name):
    return (name in self.vertex_map.keys())
  def Get_vertex(self,name):
    if name in self.vertex_map.keys():
      return (self.vertex_map[name])
  def Add_directed_edge(self, parent_name, child_name, distance):    
    if parent_name not in self.vertex_map.keys():
      self.Add_vertex(parent_name)
    if child_name not in self.vertex_map.keys():
      self.Add_vertex(child_name)
    p = self.Get_vertex(parent_name)
    c = self.Get_vertex(child_name)
    p.out_degree += 1
    c.in_degree += 1
    c.parent = p
    p.children.append(c)
    self.edge_list_map[(p,c)] = distance
  def Get_edge_length(self, parent, child):
    return (self.edge_list_map[(parent, child)])
  def Set_root(self):
    for vertex in self.vertex_map.values():
      if vertex.in_degree == 0:
        self.root = vertex
  def Get_root(self):
    if self.root == -1:
      self.Set_root()
    return (self.root)
  def Set_post_order(self):
    self.post_order_list = [self.root]
    vertices_to_visit = [self.root]
    while len(vertices_to_visit) > 0:
      v = vertices_to_visit.pop()
      vertices_to_visit += v.children      
      self.post_order_list = v.children + self.post_order_list
  def Compute_newick_format(self):
    if len(self.post_order_list) != len(self.vertex_map):
      self.Set_post_order()
    for v in self.post_order_list:
      if v.out_degree == 0:
        v.newick_label = v.name
      else:        
        c_l = v.children[0]
        c_r = v.children[1]        
        len_l = self.Get_edge_length(v,c_l)
        len_r = self.Get_edge_length(v,c_r)
        v.newick_label = f'({c_l.newick_label}:{len_l},{c_r.newick_label}:{len_r})'
    self.root.newick_label += ";"
    return(self.root.newick_label)    
  def Read_newick_string(self, newick_string):    
    rx = r'\([^()]+\)'
    hidden_vertex_ind = 1
    while "(" in newick_string:                  
      # search for the parenthesis
      m = re.search(rx,newick_string)
      # returns a tuple containing all the subgroups of the match "()"
      string_match = m.group()            
      # remove ( and )
      siblings_string = string_match[1:-1]
      c_left_name_and_length, c_right_name_and_length = siblings_string.split(",")
      c_left_name, c_left_length = c_left_name_and_length.split(":")
      c_right_name, c_right_length = c_right_name_and_length.split(":")
      if not self.Contains_vertex(c_left_name):
          self.Add_vertex(c_left_name)
      if not self.Contains_vertex(c_right_name):
          self.Add_vertex(c_right_name)
      hidden_vertex_name = "h" + str(hidden_vertex_ind)
      self.Add_vertex(hidden_vertex_name)            
      self.Add_directed_edge(hidden_vertex_name, c_left_name, float(c_left_length))
      self.Add_directed_edge(hidden_vertex_name, c_right_name, float(c_right_length))
      newick_string = newick_string.replace(string_match,hidden_vertex_name)
      hidden_vertex_ind += 1 


In [ ]:
# 1 C
print("Degree distribution")
T = Tree("test")
T.Add_directed_edge("h1","l1",0.1)
T.Add_directed_edge("h1","l2",0.3)
T.Add_directed_edge("h3","h1",0.1)
T.Add_directed_edge("h3","l3",0.5)
T.Add_directed_edge("h2","l4",0.2)
T.Add_directed_edge("h2","l5",0.2)
T.Add_directed_edge("h4","h3",0.6)
T.Add_directed_edge("h4","h2",0.3)
for v in T.vertex_map.values():
  print (f'vertex {v.name} has in-degree {v.in_degree} and out-degree {v.out_degree}')
T.Set_root()
print("\n")
print (f'The root is {T.root.name}')

Degree distribution
vertex h1 has in-degree 1 and out-degree 2
vertex l1 has in-degree 1 and out-degree 0
vertex l2 has in-degree 1 and out-degree 0
vertex h3 has in-degree 1 and out-degree 2
vertex l3 has in-degree 1 and out-degree 0
vertex h2 has in-degree 1 and out-degree 2
vertex l4 has in-degree 1 and out-degree 0
vertex l5 has in-degree 1 and out-degree 0
vertex h4 has in-degree 0 and out-degree 2


The root is h4


In [ ]:
# 1 D
T.Compute_newick_format()


'(((l1:0.1,l2:0.3):0.1,l3:0.5):0.6,(l4:0.2,l5:0.2):0.3);'

In [ ]:
# 1 E
T = Tree("T_2")
T.Read_newick_string('(((l1:0.1,l2:0.3):0.1,l3:0.5):0.6,(l4:0.2,l5:0.2):0.3);')

for v in T.vertex_map.values():
  print (f'vertex {v.name} has in-degree {v.in_degree} and out-degree {v.out_degree}')


vertex l1 has in-degree 1 and out-degree 0
vertex l2 has in-degree 1 and out-degree 0
vertex h1 has in-degree 1 and out-degree 2
vertex l3 has in-degree 1 and out-degree 0
vertex h2 has in-degree 1 and out-degree 2
vertex l4 has in-degree 1 and out-degree 0
vertex l5 has in-degree 1 and out-degree 0
vertex h3 has in-degree 1 and out-degree 2
vertex h4 has in-degree 0 and out-degree 2
